In [1]:
# importing required packages
import pandas as pd
import requests
import concurrent.futures
import numpy as np
import sys
from os.path import exists
import re
import itertools
import orjson
import time
from datetime import datetime
import urllib.parse
import os
import xml.etree.ElementTree
from bs4 import BeautifulSoup
from os.path import exists
import json
import csv
import time

In [2]:
rows = []
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [3]:
out_csv = 'indianretailer061025.csv'
input_folder = "indianretailer0610"
baseurl = "https://www.indianretailer.com/sitemap_news_page.xml"

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
    
#df = pd.DataFrame(columns=['main_category','leaf_category','ns_price','ns_series','ns_parent_brand','ns_brand','ns_product_UID','ns_model_number','ns_product_group','ns_website_prod.name','ns_displayname','ns_description','ns_features','ns_specifications','ns_extraspec','ns_image_urls','ns_url','ns_active_flag','ns_launch_date','country_of_origin','manufacturer','manufactured_details','importers_detail','importer','product_warranty','ns_video_url','pdf_url','aplus_contents'])
df = pd.DataFrame(columns=['CRA_Date', 'Topic_URL', 'name', 'language', 'Entry_Date', 'Topic', 'keywords'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

creating new CSV...


In [4]:
parent_dir = "/home/justdial/data/input"
path = os.path.join(parent_dir, input_folder) 
file_exists = exists(path)
if not file_exists :
    os.makedirs(path)
else:
    print("Path Already Exits")

In [5]:
def file_saver(url,pg):
    file_name = url.replace(' ', '_').replace('/', '').replace('.', '_').replace(':', '')
    file_name = file_name+'.html'
    print('file_name -- ',file_name)
    
    file_exists = exists(parent_dir+'/'+str(input_folder)+'/'+file_name)
    #print("getting res")

    if file_exists:
        #print("file_exists res")
        pass
    else:        
        import requests

#         cookies = {
#             'cf_clearance': '5bfoy.cGxfbxQLDzu9QrGdIpeut9yzzyrWPL_o0pgEQ-1757682864-1.2.1.1-jU3Iu0CsWy7Anz8uqv_2eVUBz84XoG0zVI8fqV_fxhj8EjLNgSC99H7vPczOhsfG2_uhdQ30CguSqkxIC8hguCXHyx8TkuX9gh3_NYh2KA3KjYKHdC5J_NDthT3ynGB4upHM1xKqoUMIC6DdhTQfpk0rXPXG5sfo0br8IN49HgVT06impPjEjiwIA9IgbFcDLOgCcf3xw4FjXtykkVBZ5T6ZoV4T6a1eS_Q4_B70Ijg',
#         }
        cookies = {
            'cf_clearance': 'aLBFTAqYJtehdwaqI03ASgqSrYeBwiKuUpdB7y9IRUY-1759734929-1.2.1.1-5ayQ7PHHSjMA1dyZ3TmZA.uMFWE6uwtexOxdbQyVkbC0atgcFTlXYYIOdh1CKLFaELuA4jTcYSAE5Fe81RDXqkbS5vSKw_YAoktSFTHx3En5lfDTAcQv6qcUpG7.DB8rxCq5p_YGQogTOVGvST.eRo6KwZSJApcDwhMFqClquSTExjtn.Vy8Y2yp.lI8dNcUEPzSNHmcwi89XhYJYsSZCUmNBoXUqCkw7.UaenHKTB8',
        }

        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            # 'Accept-Encoding': 'gzip, deflate, br, zstd',
            'Referer': 'http://localhost:8888/',
            'Connection': 'keep-alive',
            # 'Cookie': 'cf_clearance=5bfoy.cGxfbxQLDzu9QrGdIpeut9yzzyrWPL_o0pgEQ-1757682864-1.2.1.1-jU3Iu0CsWy7Anz8uqv_2eVUBz84XoG0zVI8fqV_fxhj8EjLNgSC99H7vPczOhsfG2_uhdQ30CguSqkxIC8hguCXHyx8TkuX9gh3_NYh2KA3KjYKHdC5J_NDthT3ynGB4upHM1xKqoUMIC6DdhTQfpk0rXPXG5sfo0br8IN49HgVT06impPjEjiwIA9IgbFcDLOgCcf3xw4FjXtykkVBZ5T6ZoV4T6a1eS_Q4_B70Ijg',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'cross-site',
            'Priority': 'u=0, i',
            # Requests doesn't support trailers
            # 'TE': 'trailers',
        }

        params = {
            'page': pg,
        }

        response = requests.get('https://www.indianretailer.com/sitemap-news-new.xml', params=params, cookies=cookies, headers=headers)
#         response = requests.get(url)
        if response.status_code == 200:
            with open(parent_dir+'/'+str(input_folder)+'/'+file_name,'w') as f:
                
                now = datetime.now()
                tmts = str(now.strftime("%Y-%m-%d:%H-%M-%S"))
                response = "\n\n__xxtmtstart"+tmts+'__xxtmtsend\n\n'+response.text
                print('tts', tmts)
                
                f.write(response.encode(encoding="utf-8",errors='ignore').decode())
                print("File saved...", file_name)
    HTMLFile = open(parent_dir+'/'+str(input_folder)+'/'+file_name, "r")
    gpage = HTMLFile.read()
    gpage = re.sub('\\n','',str(gpage))
    gpage = re.sub('<!--(.*?)-->','',str(gpage))
    gpage = re.sub('#038;','',str(gpage))
    return gpage

In [6]:
def parse_product(ppage):
    ppage = re.sub('<!--(.*?)-->','',str(ppage))
    ppage = re.sub('&times;','x',str(ppage))
    ppage = re.sub('amp;','',str(ppage))
    ppage = re.sub('&#8211;','-',str(ppage))
    ppage = re.sub('&#8217;','',str(ppage))
    ppage = re.sub('&#038;','&',str(ppage))
    ppage = re.sub('&#8243;','Inches',str(ppage))
    #ppage = re.sub('&quot;','Inches',str(ppage))
    ppage = re.sub('&nbsp;','',str(ppage))
    
    main_cut = re.findall('<urlset.*?>(.*?)</urlset>',str(ppage))[0]
    
    main_list = re.findall('<url>(.*?)</url>',str(main_cut))
    print('total records ==',len(main_list))
    
    for data in main_list:
        #url
        try:
            url = re.findall('<loc>(.*?)</loc>',str(data))[0]
        except:
            url = ''
            
        #name
        try:
            name = re.findall('<news:name>(.*?)</news:name>',str(data))[0]
        except:
            name = ''
            
        #language
        try:
            language = re.findall('<news:language>(.*?)</news:language>',str(data))[0]
        except:
            language = ''
            
        #publication_date
        try:
            date_cut = re.findall('<news:publication_date>(.*?)</news:publication_date>',str(data))[0]
            date = re.findall(r"(\d{4}-\d{2}-\d{2})",date_cut)[0]
        except:
            date = ''
            
        #title
        try:
            title = re.findall('<news:title>(.*?)</news:title>',str(data))[0]
        except:
            title = ''
            
        #keywords
        try:
            keywords = re.findall('<news:keywords>(.*?)</news:keywords>',str(data))[0]
        except:
            keywords = ''
            
            
    # product_name
    
    # description
    
    # specification
    
    # images
    
    # pdfs
    
        # timestamp
        ttms = re.findall('__xxtmtstart(.*?)__xxtmtsend', ppage)[0]

#         uid = '' + sub_name[0:2] + product_name
#         uid = uid.lower().replace(' ','')
#         print('uid -- ',uid)

#         display = '' + ' ' + product_name + ' ' + sub_name
#         print('display --- ',display)

        my_dict = {}
        my_dict['CRA_Date'] = ttms
        my_dict['Topic_URL'] = url
        my_dict['name'] = name
        my_dict['language'] = language
        my_dict['Entry_Date'] = date
        my_dict['Topic'] = title
        my_dict['keywords'] = keywords
        
#         my_dict['ns_description'] = description.strip()
#         my_dict['pdf_url'] = documentation
#         my_dict['ns_specifications'] = specification
#         my_dict['ns_image_urls'] = images
#         my_dict['ns_url'] = product_url
#         my_dict['ns_features'] = features

        df = pd.DataFrame(my_dict, index=[0])
        list1=list(my_dict.values())
        print(f"writing to csv .... ")
        #count+=1
        with open(out_csv, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=header_list,)
            if file.tell() == 0:
                writer.writeheader()# Write the data row
            writer.writerow(my_dict)
        print('done')

In [7]:
def fetch_products(main_url):
    i = 1
    if re.match('http',str(main_url)):
        pass
        #print('okay')
    else:
        new_main_url = str(baseurl) + main_url
    
    print('main_url --- ',main_url)
    pg = i+1
    ppage = file_saver(main_url,pg)
    parse_product(ppage)
    
#     product_regex = '<div class="post-img-wrap">\s*<a href="(.*?)".*?>'
#     product_list = re.findall(product_regex,str(spage))
#     print('product_list --- ',product_list)
#     print(len(product_list))
    
#     for i in product_list:
#         product_link = i
#         print('product url ---',product_link)
#         ppage = file_saver2(product_link)
#         parse_product(ppage,product_link,main_name)
        
#     next_pager_cut = re.findall('<collection-pagination-item>(.*?)</collection-pagination-item>',str(spage))
#     print('====================================================================')
#     print(len(next_pager_cut))
    
#     if len(next_pager_cut) > 0:
#         next_page_list = re.findall('<a href="(.*?)" class="btn js-append-content" aria-label="Next">',str(next_pager_cut))
#         for i in next_page_list:
#             print(i)
#             next_url = baseurl  + i
#             print('next page url -- ',next_url)
#             fetch_products2(main_name,next_url)
            

In [8]:
file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html')
if file_exists:
    pass
else:
    
    import requests
    
    cookies = {
        'cf_clearance': 'aLBFTAqYJtehdwaqI03ASgqSrYeBwiKuUpdB7y9IRUY-1759734929-1.2.1.1-5ayQ7PHHSjMA1dyZ3TmZA.uMFWE6uwtexOxdbQyVkbC0atgcFTlXYYIOdh1CKLFaELuA4jTcYSAE5Fe81RDXqkbS5vSKw_YAoktSFTHx3En5lfDTAcQv6qcUpG7.DB8rxCq5p_YGQogTOVGvST.eRo6KwZSJApcDwhMFqClquSTExjtn.Vy8Y2yp.lI8dNcUEPzSNHmcwi89XhYJYsSZCUmNBoXUqCkw7.UaenHKTB8',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://www.indianretailer.com/sitemap_news_page.xml?__cf_chl_tk=Vebo8ZZYWdeIsk6pTdKJTc5C6GE5_hZCA9zanS9wtg8-1757674650-1.0.1.1-EA1VU0Bg6SQAi34MvOBAF5V8jYD8jDqaYuChD1GiAQY',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Origin': 'https://www.indianretailer.com',
        'Connection': 'keep-alive',
        # 'Cookie': 'cf_clearance=5bfoy.cGxfbxQLDzu9QrGdIpeut9yzzyrWPL_o0pgEQ-1757682864-1.2.1.1-jU3Iu0CsWy7Anz8uqv_2eVUBz84XoG0zVI8fqV_fxhj8EjLNgSC99H7vPczOhsfG2_uhdQ30CguSqkxIC8hguCXHyx8TkuX9gh3_NYh2KA3KjYKHdC5J_NDthT3ynGB4upHM1xKqoUMIC6DdhTQfpk0rXPXG5sfo0br8IN49HgVT06impPjEjiwIA9IgbFcDLOgCcf3xw4FjXtykkVBZ5T6ZoV4T6a1eS_Q4_B70Ijg',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Priority': 'u=0, i',
        # Requests doesn't support trailers
        # 'TE': 'trailers',
    }

    data = {
        'db3d1d212a3d6799bce40979b407d94b72053667f71ae191434d5cd82e6c1117': 'ECYHHqpfRIQlPGI9pT.Jm0EqaOduFGAiCRBNeNi2T80-1757674650-1.2.1.1-xzQ4.sLdwWnMB09mFa.j1Fai2A5XuAGW_ejKMGNnetMrQAOllF8TqXWldINY7jY8.S6NpcjA4DCJVGoiGhLkJMwUrkXsHuaX3hZ_qj1QfUhEanyqsba5IHOm7z6weoLsJuQ9NAtVT8excsBpqGasMVkYzgx06poCTmngZeWgedPmpWO7Dda1e9D9YaitERH1Ff2gIIaHjBvuGunw2iywZQ6TmHgEiGrZOfxxhYkW2J1dDUhOYYegAy9KXXUYCdV_9F42WQ_kyzHN5Ok44VYx1vmJZntvrbbCY4BTZAJOrd0mnJttmSfDBjEFLiUURtcJWs4Ne8NGkzJ_f8JbG6ZsZAC_j9E_fRf6Hb2KNkvI9x42y0WJOinqwcZKeJC8g7s2isYBFoK3f2ZdAGOhwk6PAJ3l6ye4WRCCqGaVZEdfs6ExPbZr_M2gFQTaWpnVNFpAwnWhRb2oRs4GV_2E8xYreGli91GkO4TwkDJ_CMaEFvxr9RGKdbwzYe5syyTodasl.oZ_7ye7fLNVmYlxcO__4S3lYdKL.T2UiRQDoa6ZiRJHiYUF0R9TnqfJ5iGinUXwBtPLTyKnt82vKgxztMW4iIM_IAliGg_Bsm9qpReGTyIhaCvmptbRCX5mUknNL4I0e_ydImMoY2QDvTmv__I1zbmeKVOxg94_8QGQgOPqusve78k5nsywggiTPH19_dOgo7bWmzjF4RfPouN_Pc3yXQxtLEhM06Qe8tc3H0nC6SQ0aOCNgeFpqpfLcXjFUibCPaE5TVmDNZkSwpL2DY1tx7JmzcVlswchlrQ1oMjR66r5fDgUxyCt5uU388O1fk7z0i2HLiYTW7WFscrzff.fLmFN2fytqeq6AUI5dXws5UPurV_ifS1IH2ehZ3_JlzvkQIb3s.SmN61WXMMoH2rKTf1_btCJXil_EOhxMFbbqu2eZph9KVS_vdJuBcG4KubDkL5XMiJZzC2hBnx6v1zlpxqdO5QuLyRFaeIpM7ttNmmUaXkjuMRHu8ut8bIU5POK',
        '6711bf024a67e6d818665f1b49e0ff761ae479e9cf3bb40342be9a7b9c2841b2': 'SLMTTYYNXcsULEVBRrWNEgnQUC6x1G9uL044ii3ouNE-1757674650-1.2.1.1-GKWdLF51AYxFXGe_cpncxGg0o7S.YYt1xEVWc5wmu2wlwExdLiLRVCShfp9HJuv4E37XSIlhACcCCgLSnc0evE0SSLl.erJdDREcFLCVyFFxUusQQErAYLDSn4y_p6fsaEM1o2AzCejEts8bjVOXr4V_I1bds9xxl2Zq0UViwvo8FfFDmjdMJWobNjj.v0Hi5f.Ie9hBPgAKhwHCZ4WsrSWt.Ai5wOyhtfr_3rCw3xvnTXJWoeyUJxYCAgavytPjAsWE23u0z5bnNXr_hHIgusUoQEStIZjryhQ7QY7L4YlYjPTwAqqyMM7YQrVHh3oMP.R2VEEu9mUbzMvSNM_WA7XeecykcvxlblO4R5nIXNCiAh_z6Q25ZOLQDXWoq5Nk24XqmFP6IayL08xdSIgdS7_4npG2HRdTowZX.tI_sk7H6R2VshF32w9CgjOmJi28sIttRfvAdYjHq5UEGOkasa3espZuELl8y.ebJazhxPv9zNerHil5guTQjL0sSAv68Ws6eefi3.sYCYD4yTIjf0uA.zq5WolOmSyAEXQrYtv84xrnN7qOfASr.2FcYm2jVdEPnx7TLPt8N6aAeNZLnoel7a6IdAPEQFhABxNWbh6z2lTRiMUHfFTjxE2KuSWF9yWbfYaB2rER.g9RggCY9KxPqARCYYyvSEqCUCr_eZtEzg76vIwKoflxLatdFO2R_rWk3W4d07oz4aR1khpj5o9Lcx7qoCXqEwvr_Vk3.H9gqgSEHD9i9fMppNapNkC4snhPkyM6S9i7ty5GkmKU2yTHv_m71RgWeqbJY9d_Lnjh.fPlQKCtP1zHDuSSKynVl1UGYCoYWQWxPSlRZ6x0.it7Ebpww056Y9o.HFabDzHlZAVqhqugWK4mCb.ADJoqjU9Oqwa91_SERikDXkZWr6keRQlZLPmt52QZcPcywqIQ2Wpz7tD979a_duzkxBSk4pX4AuSM7KF1Tt8hleIW4vv38p6dNJKmgCO1G1yGpefzPu54h6N44S5h0t52DrqvyPnpW5Eq2o6dvxue19gmSjHmAXAm8N69SNSoqkN0LR_APNPTnLm8XtORgrWfh_P5dtdrRWMbWhld1dm3cs2mstM_cuf0Cga1AkWUz6jBcgMtbZaK4eg3Zy1fzJLSlp.duw.SoLEbda.rgqrkbqglT9BgLmutz_Q9ziFLVb0ueK5u1iSwxkCwZqMJRYQ5ZNXSSFDKZZFF4vZdyCPOrmZKNzV0Dfm1Z3TodvPHPMpZdpRF2YwxFnkA1dC1hCK4zSkPihPY8eg02H5dZAjDSQQHvofD.Q2bwkJQczxqmMo.65s_oGwWMjoWdkGE8JMf1we0iESfhxIjWEoJ1Zs5A9RLEaO4J7SAT5miFYEF7MCJxCo2grutK9fBYiVeJWNcSplnLWHB5ew5zDVYlcTdZmNSt0iJ3UqteLyGX3e51ZSTX6IycL2Ia6HvDg4JOsVzqKxBFbTnqGmNTDhhokq8vkpQJ5gmYCu0rFT6Dkg7VyMtleLj5bpCMY_I_KrTVQ0HZsQz6jsuvrx3Hd4NYomULfpEq_29qXGUAXNx0f6MvSVjZVs5vf6k0uCfDPtsC4L_TgCSRDCEALfDQ5q7ePJpPNgMkosztb26k.SRp2Pu2aoJFbMnmq1RpeLyq4w1qUg92.pbUOJlcuyPaUhKTd7bD_vt7M2s9yK4JaUdPhjoM9RrPJJoRGBxcSvEYSq35vthBSBhnZL2wM_nLg2FaJMunQRCuwj8GIEQ2Of.mBz4Nco3m82f9Jg4.2IyiRFWI97xoweXkTn6NnX8E.C3DZPwf7cXur8GCkWaFrEkdCHVcyhU3oY4CwGZwLdd03SikwLEbMwn5HX7D8dnhtzQ.2hZAKheoemrwCHfr_mNcvgxpBDTpsKbt44GeriPz2EHWN1Suk13yFLtSFb9tc9mPmMGuu7J5i_Quy9fL.RsWUAYqJRLWVVEjvEb374NhBjFP1OH8LjpLYs6gYD5pXOtri.5tEW82HBvCW9xZzr0ygehaJCqS3b8mS9CMR4rcHyPmzcqIogaCm2d.J.JuHLPTOg8rj5D59ARr46GFluv52LrBFtPmfTaTNcvb9a_ZJTxD25NsrqsWB2f2dTdQwKdq1ViKci67L8Hj1v23OaQddTNjeNkDSENMfxUqcvDGVyZnPMmv6sHrMwgmtKPTS90PEONIy.L8J3_LFhsdR3Mam51KDYAYRCjBKZB7I69YBypSiFgjXqrS8CbKeTKahFwVOvxbwqjwGLyQcuAzVUE1Tv0K8U3xCX5x4wiqNhZ6xlQJscjVqNEoRBmkKQ2vvdgulbqzasDGQdyMl8xD5fXcQM9diVhSWIQxCBOutBnqpeuIIt5zhIdNnerJEefcfYhO8lTygpzhBNx69qrBoV9hNFEy_w0IN.qgqD6eOxN3jPO0oh2TOyuxih9gY3KX7YevJK98cQY26ioVQf4dkyl6W9It2vuSS5C8ARdU8Q1B_2jdMkLuD24q.POBxSdt9yNOCJ_6aDwUdAc3Fe67eCtG3cj.vU2.VUaOc.XmNgeB4V9ZUF.GWJfknECPKwQF3jInR4YvrFFE5mIw6uOEz3ice5t2Cn_ShJpaJIumwDanZl.emt3aON8RgxtcTJ_ECGOpxJ4siOnx48UwcM9dBjTKZ9OSR4LtZzPb7hyUp0bYJOlfJ0qFY9GXlkfFHN7l5XL4Frx6RA5CXC99DvLkYnEyB28v2vZIrPkZkUeqKDdPcfRP2zgPf47vPtwYxFQs1uvNjFwjwF2QDSONQ9mBdJYHBxe0Y79xam0hMonLii5snElytUwAYRdLH7EsbsE6k2wPw0Jb70vQbvoMuQQ3SM4bBz3FH5VPEQQCaQDneScehl8fEexlhhYxVXDMHsN9Qyg721y7t6mZ4dQmnz7KwP4hMIBTv3K1fJ2DjR06l1YGeSjsiApffsMxKMNdogzgSpLvYmYSyYaJasAcb.A7rTmrs8r1sOWK2wDpPUp0hJdCHCvlIcfPikZtrKKGTLUxyp6hQNoUkqN.48nUZjPHIdIiw72u3_2g_UkmjGd.4JzZ1pnswyBhqX6w1bhhCckE9d6H_Pp6_LtezvLcOZzb.ZbJ3O2sL_43z7GcKK5nv4Iat2g.QRT.i8rNQoeryR5HpOLyturUkct6yhlgxoLNd6A67jX5.xWt0Dx09I5nV5X_ZRIythYfL3LRsVI0cFY5b4V9LWlsj.98oS7CiJKcerIyMQr7R2V8gSuRTQNMwP._H6qhjV5wwzDYwv.4w9GtmjIVckiOIwOm7.2PGrDILZjfK8qiX0TxyaCsO_ci_GeFRFfhRqQ94nmp3ky85rj05PsuNjYac3yDSAdtZ4clTyhMMHh3KXyu4_E7TOb5hnZd8X6xr3RPOSFVbk5VKKNIQCbrgyamU4zWFeEMMN2vzibi.KwX5XSYjse5sumRxK1XPC0apT5yWRBEbCIuV3L7IYOtsN81LFWuiplcKUIXksr3DKlPpJupITlS8o2p_6AFdqnhBOcrAg9XavpMLcQS5zQ.wv0EUmf5ID_DNGOk8Fuen0RKN7D5J5j6XsWp5xqNESD2utihvUoFhNsTEAJr3u0lRC0dJS9cVnFuJwt95_V5i9U0dFj8pdcDp7J6ea12GqOwXE6QFofCl5AVTtQeJZh1ZdMSSbgm94WftAxC4FmPjr3JsqUkm1ZSK192BMgEDWYE93fU0353wV3b_2SK_jCeQdRBbZHZe6wXsg.Up1K2wqlYkUYDq0CDaCCsQ8foMyW8L_ipa8p0DcCthxDqwdwDzwe8eGBcLCNm3UcU06_hnxcIcO2QlNAoYRzQgbU2XEgBQOdUlUXikC7NE2C7q76YNSr0siJEKt4Gkkf0V9rn9c3SXoDM06pptPCH7vL8mHVuSgEzkcGcdBZUumq2Vp9tozpn.1anG7HegmF9MBtjc2NpV2C9_3EWpOobulpU26bJXCBMdKNsshBLKEBMM_nX2kQmRxIR_dFIWYJjTLM9R0wxBJ849uAJ6QhE1f3dDEO39Unulgs.i5GfnYeQcSUYkwXo2fEAb9B4Kr5UV9NeOrP96HQJngymDFI.XtP9r3XGU1nBID8H4L09PWv4OITTVmsKURwq41kIMMOdtWow4YtYYCjTs0_LvsBTiiD3tOSHglC6eatas3qTTDNRZZ6.OtS6a7jI4T4fPVdsnv.5uP0bDitqNF7Y1GY1HzxqDDsjtFGBpfAVFyJ.blSyJZnyDipQcCytbW8LbPDkKBQ0IBNs5JxXFOXGzPRbbsGdZPKVjg8FGqosWeJnA7cXh8RyFvPHyUAddJ_pDa.4sYT258yMNkmftKDcN763lvcoPkpg8DbX_rp9QYrOaESWfEokW.rjBvwhfIwQmS0r4.ABf4x0y8ji0p_jgJyn.0fnwk7vE6JOUx2TXE3Eey5oLV2BbG2RLtiugJXpQKqBCTDRuwUl.bfzcBNNYIKkocHuVOxQFQUEHX3',
        '0eb751175e38c4620d78d037dff0841da3e61f914e1b4e793a641a8028fffbbb': 'XiCHK4f3RW8LJCwcFMRO3k12uMLfIW4MuuaUn2iXL84-1757674680-1.0.1.1-S.CsfKa7nVxCraLs1bu7rgBX5aiE.G5a3iBqt.2jBL.f2gBSXZWnBxvxOobpMSWZmjFfrlo7JzuZOUTDibBbxwPUIyRKjKPwgwGCZZfPQcwhBlh6JKpVGt8MBETyHXES2tKxtWN41yFwVLZSgji0scVzkAneH.mvhbCzqdh20ZrGiCHwwp1HlER3sAANavCq',
    }

    homepage = requests.post('https://www.indianretailer.com/sitemap_news_page.xml', cookies=cookies, headers=headers, data=data)

#     homepage = requests.get(baseurl)
    print(homepage.status_code)
    if homepage.status_code == 200:
        with open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html','w')as f:
            
            now = datetime.now()
            tmts = str(now.strftime("%Y-%m-%d:%H-%M-%S"))
            hpage = "\n\n__xxtmtstart"+tmts+'__xxtmtsend\n\n'+homepage.text
            print('tts', tmts)
                
            f.write(homepage.text.encode(encoding="utf-8",errors='ignore').decode())
                
HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html', "r")
hpage = HTMLFile.read()
hpage = re.sub('\\n','',str(hpage))
hpage = re.sub('<!--(.*?)-->','',str(hpage))
hpage = re.sub('#038;','',str(hpage))
# print(hpage)

main_cut = re.findall('<sitemapindex.*?>(.*?)</sitemapindex>',str(hpage))

main_list = re.findall('<loc>(.*?)</loc>',str(main_cut))
print()
print(main_list)
print(len(main_list))

# m_hash = re.findall('',str(main_cut))
# main_cat_dict = dict(m_hash)
# #print(main_cat_dict)

for main_url in main_list:
    print('main_url --- ',main_url)
#     main_name = main_cat_dict[main_url]
#     print('main_name --- ',main_name)
    fetch_products(main_url)

200
tts 2025-10-06:12-48-10

['https://www.indianretailer.com/sitemap-news-new.xml', 'https://www.indianretailer.com/sitemap-news-new.xml?page=1', 'https://www.indianretailer.com/sitemap-news-new.xml?page=2', 'https://www.indianretailer.com/sitemap-news-new.xml?page=3', 'https://www.indianretailer.com/sitemap-news-new.xml?page=4', 'https://www.indianretailer.com/sitemap-news-new.xml?page=5', 'https://www.indianretailer.com/sitemap-news-new.xml?page=6']
7
main_url ---  https://www.indianretailer.com/sitemap-news-new.xml
main_url ---  https://www.indianretailer.com/sitemap-news-new.xml
file_name --  httpswww_indianretailer_comsitemap-news-new_xml.html
tts 2025-10-06:12-48-15
File saved... httpswww_indianretailer_comsitemap-news-new_xml.html
total records == 2000
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 


done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

...

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing to csv .... 
done
writing

In [9]:
for i in range(1,6):
    print(i)

1
2
3
4
5


In [ ]:
import requests

cookies = {
    'cf_clearance': 'kxOO_oRDKZTXes5l2.AsbEUzwBhXPYBhSy5nrVqodh4-1757917370-1.2.1.1-UaiT6DifT4o9mUWZowM0YyRis8v5kkLHHTzy5aFUTuduWynoLZkNnUqdZHTwTdVi0GiRZJSc_ym7.d3Q0qo.268NnWcH_GwmCCHhcHdFTKW2JKPd97qcspydtlhYhx_uYgT52euuOhlQQ5EIjvtCU7Q4Q4tz4ABnJEtQWQpx.S7rFcuw6.btTOIQCdiffmiRymqoFuUxJnC0HpUKPUp.Thmhot7iTuq7pDCbI3qg92o',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Referer': 'https://www.indianretailer.com/sitemap_news_page.xml',
    'Connection': 'keep-alive',
    # 'Cookie': 'cf_clearance=kxOO_oRDKZTXes5l2.AsbEUzwBhXPYBhSy5nrVqodh4-1757917370-1.2.1.1-UaiT6DifT4o9mUWZowM0YyRis8v5kkLHHTzy5aFUTuduWynoLZkNnUqdZHTwTdVi0GiRZJSc_ym7.d3Q0qo.268NnWcH_GwmCCHhcHdFTKW2JKPd97qcspydtlhYhx_uYgT52euuOhlQQ5EIjvtCU7Q4Q4tz4ABnJEtQWQpx.S7rFcuw6.btTOIQCdiffmiRymqoFuUxJnC0HpUKPUp.Thmhot7iTuq7pDCbI3qg92o',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=0, i',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

response = requests.get('https://www.indianretailer.com/sitemap-news-new.xml', cookies=cookies, headers=headers)


In [ ]:
import requests

cookies = {
    'cf_clearance': 'kxOO_oRDKZTXes5l2.AsbEUzwBhXPYBhSy5nrVqodh4-1757917370-1.2.1.1-UaiT6DifT4o9mUWZowM0YyRis8v5kkLHHTzy5aFUTuduWynoLZkNnUqdZHTwTdVi0GiRZJSc_ym7.d3Q0qo.268NnWcH_GwmCCHhcHdFTKW2JKPd97qcspydtlhYhx_uYgT52euuOhlQQ5EIjvtCU7Q4Q4tz4ABnJEtQWQpx.S7rFcuw6.btTOIQCdiffmiRymqoFuUxJnC0HpUKPUp.Thmhot7iTuq7pDCbI3qg92o',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Connection': 'keep-alive',
    'Referer': 'https://www.indianretailer.com/sitemap_news_page.xml',
    # 'Cookie': 'cf_clearance=kxOO_oRDKZTXes5l2.AsbEUzwBhXPYBhSy5nrVqodh4-1757917370-1.2.1.1-UaiT6DifT4o9mUWZowM0YyRis8v5kkLHHTzy5aFUTuduWynoLZkNnUqdZHTwTdVi0GiRZJSc_ym7.d3Q0qo.268NnWcH_GwmCCHhcHdFTKW2JKPd97qcspydtlhYhx_uYgT52euuOhlQQ5EIjvtCU7Q4Q4tz4ABnJEtQWQpx.S7rFcuw6.btTOIQCdiffmiRymqoFuUxJnC0HpUKPUp.Thmhot7iTuq7pDCbI3qg92o',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=0, i',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

params = {
    'page': '2',
}

response = requests.get('https://www.indianretailer.com/sitemap-news-new.xml', params=params, cookies=cookies, headers=headers)